In [1]:
import numpy as np
import os
import pickle as p
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import networkx as nx
import sys
import slam.io as sio
import networkx as nx
import tools.graph_visu as gv
import tools.graph_processing as gp
import matplotlib.pyplot as plt
import random
import plotly.express as px
import plotly.figure_factory as ff
import trimesh
import slam.topology as stop
from sphere import *
import random

In [7]:
def generate_sphere_random_sampling(vertex_number=100, radius=1.0):
    """
    generate a sphere with random sampling
    :param vertex_number: number of vertices in the output spherical mesh
    :param radius: radius of the output sphere
    :return:
    """
    coords = np.zeros((vertex_number, 3))
    for i in range(vertex_number):
        M = np.random.normal(size=(3, 3))
        Q, R = np.linalg.qr(M)
        coords[i, :] = Q[:, 0].transpose() * np.sign(R[0, 0])
    if radius != 1:
        coords = radius * coords
    return coords

In [8]:
def tri_from_hull(vertices):
    """
    compute faces from vertices using trimesh convex hull
    :param vertices: (n, 3) float
    :return:
    """
    mesh = trimesh.Trimesh(vertices=vertices, process=False)
    return mesh.convex_hull

In [4]:
def generate_noisy_graph(original_graph, nb_vertices, sigma_noise_nodes = 1, sigma_noise_edges = 1, radius = 100):
    
    # generate ground_truth permutation
    ground_truth_permutation = np.random.permutation(nb_vertices)
    #ground_truth_permutation = np.arange(nb_vertices)
    
    # create a new graph
    noisy_graph = nx.Graph()

    # add the nodes (not very clean but it works fine and run in no time)
    for node_to_add in range(len(ground_truth_permutation)):
        for original_node, current_node in enumerate(ground_truth_permutation):
            if current_node == node_to_add:
                print(original_node,node_to_add)
                noisy_coordinate = original_graph.nodes[original_node]["coord"] + \
                    np.random.multivariate_normal(np.zeros(3), np.eye(3) * sigma_noise_nodes)

                # We project on the sphere
                noisy_coordinate = noisy_coordinate / np.linalg.norm(noisy_coordinate) * radius
                noisy_graph.add_node(node_to_add, coord = noisy_coordinate)
        
    compute_noisy_edges = tri_from_hull(list(nx.get_node_attributes(noisy_graph,'coord').values())) # take all peturbated coord and comp conv hull.
    adja = stop.edges_to_adjacency_matrix(compute_noisy_edges) # compute the new adjacency mat
    edge_list = [tuple((u,v)) for u,v in zip(adja.nonzero()[0],adja.nonzero()[1])] # convert to edge list to add edge attributes.


    # add the edges
    for edge in edge_list:

        # get the original and corresponding ends
        #end_a_corresponding, end_b_corresponding = ground_truth_permutation[edge[0]], ground_truth_permutation[edge[1]]
        coordinate_a, coordinate_b = noisy_graph.nodes[edge[0]]["coord"], noisy_graph.nodes[edge[1]]["coord"]

        # calculate noisy geodesic distance
        noisy_geodesic_dist = gp.compute_geodesic_distance_sphere(coordinate_a, coordinate_b, radius)

        # Add the new edge to the graph
        noisy_graph.add_edge(edge[0],edge[1], weight = 1.0, geodesic_distance = noisy_geodesic_dist)
    
    return ground_truth_permutation, noisy_graph

In [5]:
def generate_noisy_graph_2(original_graph, nb_vertices,nb_outliers,sigma_noise_nodes=1, sigma_noise_edges=1, radius=100):
    # Perturbate the coordinates
    # noisy_coord = [points+np.random.multivariate_normal(np.zeros(3), np.eye(3) * sigma_noise_nodes)
    # 	for points in list(nx.get_node_attributes(original_graph,'coord').values())]

    ground_truth_permutation = np.arange(nb_vertices)
    # create a new graph
    noisy_graph = nx.Graph()
    # add the nodes (not very clean but it works fine and run in no time)
    for node_to_add in range(len(ground_truth_permutation)):
        for original_node, current_node in enumerate(ground_truth_permutation):
            if current_node == node_to_add:
                # noisy_coordinate = original_graph.nodes[original_node]["coord"] + \
                # 	np.random.multivariate_normal(np.zeros(3), np.eye(3) * sigma_noise_nodes)

                # Sampling from Von Mises - Fisher distribution
                original_coord = original_graph.nodes[original_node]["coord"]
                mean_original = original_coord / np.linalg.norm(original_coord)  # convert to mean vector
                noisy_coordinate = Sphere().sample(1, distribution='vMF', mu=mean_original,
                                                   kappa=sigma_noise_nodes).sample[0]

                #noisy_coordinate = noisy_coordinate / np.linalg.norm(noisy_coordinate) * radius
                noisy_coordinate  = noisy_coordinate * np.linalg.norm(original_coord)
                noisy_graph.add_node(node_to_add, coord=noisy_coordinate)
                
    noisy_coord = list(nx.get_node_attributes(noisy_graph, 'coord').values())
                
    # Add Outliers
    if nb_outliers > 0:
        print("nb_outliers: ",nb_outliers)
        sphere_random_sampling = generate_sphere_random_sampling(vertex_number=nb_outliers, radius=radius)
        # merge pertubated and outlier coordinates to add edges 
        all_coord = noisy_coord + list(sphere_random_sampling)
    else:
        all_coord = noisy_coord

    print("nb_outliers: ",nb_outliers)


    compute_noisy_edges = tri_from_hull(all_coord)  # take all peturbated coord and comp conv hull.
    adja = stop.edges_to_adjacency_matrix(compute_noisy_edges)  # compute the new adjacency mat.

    # Extracting the ground-truth correspondence
    ground_truth_permutation = []
    for ar1 in compute_noisy_edges.vertices.view(np.ndarray):
        index = 0
        for ar2 in noisy_coord:
            if np.mean(ar1) == np.mean(ar2):
                ground_truth_permutation.append(index)
                index += 1
                break
            else:
                index += 1
                continue

    print("Total Ground truth nodes: ", len(ground_truth_permutation))
    print("Total number of nodes : ", len(compute_noisy_edges.vertices))

    noisy_graph = nx.from_numpy_matrix(adja.todense())

    node_attribute_dict = {}
    for node in noisy_graph.nodes():
        node_attribute_dict[node] = {"coord": np.array(compute_noisy_edges.vertices[node])}

    nx.set_node_attributes(noisy_graph, node_attribute_dict)

    nx.set_edge_attributes(noisy_graph, 1.0, name="weight")

    edge_attribute_dict = {}
    id_counter = 0  # useful for affinity matrix caculation
    for edge in noisy_graph.edges:
        # We calculate the geodesic distance
        end_a = noisy_graph.nodes()[edge[0]]["coord"]
        end_b = noisy_graph.nodes()[edge[1]]["coord"]
        geodesic_dist = gp.compute_geodesic_distance_sphere(end_a, end_b, radius)

        # add the information in the dictionnary
        edge_attribute_dict[edge] = {"geodesic_distance": geodesic_dist, "id": id_counter}
        id_counter += 1

    # add the edge attributes to the graph
    nx.set_edge_attributes(noisy_graph, edge_attribute_dict)

    return np.array(ground_truth_permutation), noisy_graph

In [2]:
def generate_noisy_graph_3(original_graph, nb_vertices,nb_outliers,sigma_noise_nodes=1, sigma_noise_edges=1, radius=100):
    # Perturbate the coordinates
    
    noisy_coord = []
    key = [] 
    value = []
    
    for index in range(nb_vertices):
        
        # Sampling from Von Mises - Fisher distribution
        original_coord = original_graph.nodes[index]["coord"]
        mean_original = original_coord / np.linalg.norm(original_coord)  # convert to mean vector
        noisy_coordinate = Sphere().sample(1, distribution='vMF', mu=mean_original,
                                           kappa=sigma_noise_nodes).sample[0]

        noisy_coordinate  = noisy_coordinate * np.linalg.norm(original_coord)
        #print(noisy_coordinate)
        noisy_coord.append(noisy_coordinate)
                   
    # Add Outliers
    sphere_random_sampling = []
    if nb_outliers > 0:
        print("nb_outliers: ",nb_outliers)
        sphere_random_sampling = generate_sphere_random_sampling(vertex_number=nb_outliers, radius=radius)
        # merge pertubated and outlier coordinates to add edges 
        all_coord = noisy_coord + list(sphere_random_sampling)
    else:
        all_coord = noisy_coord

    print("nb_outliers: ",nb_outliers)

    compute_noisy_edges = tri_from_hull(all_coord)  # take all peturbated coord and comp conv hull.
    adja = stop.edges_to_adjacency_matrix(compute_noisy_edges)  # compute the new adjacency mat.

    # Extracting the ground-truth correspondence

    noisy_graph = nx.from_numpy_matrix(adja.todense())
    
    print("all nodes: ",len(noisy_graph.nodes))

    node_attribute_dict = {}
    for node in noisy_graph.nodes():
        node_attribute_dict[node] = {"coord": np.array(compute_noisy_edges.vertices[node])}

    nx.set_node_attributes(noisy_graph, node_attribute_dict)

    nx.set_edge_attributes(noisy_graph, 1.0, name="weight")

    edge_attribute_dict = {}
    id_counter = 0  # useful for affinity matrix caculation
    for edge in noisy_graph.edges:
        # We calculate the geodesic distance
        end_a = noisy_graph.nodes()[edge[0]]["coord"]
        end_b = noisy_graph.nodes()[edge[1]]["coord"]
        geodesic_dist = gp.compute_geodesic_distance_sphere(end_a, end_b, radius)

        # add the information in the dictionnary
        edge_attribute_dict[edge] = {"geodesic_distance": geodesic_dist, "id": id_counter}
        id_counter += 1

    # add the edge attributes to the graph
    nx.set_edge_attributes(noisy_graph, edge_attribute_dict)
    
    ground_truth_permutation = []

    for ar1 in noisy_coord:
        for i in range(len(noisy_graph.nodes)):
            if np.mean(ar1) == np.mean(noisy_graph.nodes[i]['coord']):    
                if i >=20:
#                     print("Key",i)
                    key.append(i)
                ground_truth_permutation.append(i)
                break
                
    for outlier in sphere_random_sampling:
        for i in range(len(noisy_graph.nodes)):
            if np.mean(noisy_graph.nodes[i]['coord']) == np.mean(outlier):
                if i<20:
                    value.append(i)
#                     print("value",i)
                
    if nb_outliers > 0:
        index = 0
        for j in range(len(ground_truth_permutation)):
            if ground_truth_permutation[j] == key[index]:
                ground_truth_permutation[j] = value[index]
                index+=1
                if index == len(key):
                    break
                    
    key = key + value
    value = value + key
    
    mapping = dict(zip(key,value))
    print(mapping)

    noisy_graph = nx.relabel_nodes(noisy_graph, mapping)

    return ground_truth_permutation,noisy_graph,noisy_coord

In [19]:
G = nx.read_gpickle("reference_0.gpickle")

In [20]:
original_coord =  list(nx.get_node_attributes(G,'coord').values())

In [9]:
#ground_truth_permutation_old, noisy_graph_old = generate_noisy_graph(G,len(original_coord),10,0)

In [10]:
# noisy_graph_old_coord = list(nx.get_node_attributes(noisy_graph_old,'coord').values())
# noisy_graph_old_coord[:5]

In [11]:
#ground_truth_permutation_old

In [12]:
# ground_truth_permutation, noisy_graph = generate_noisy_graph_2(G,len(original_coord),0,10000,0)

In [13]:
#original_coord

In [14]:
# noisy_graph_coord = list(nx.get_node_attributes(noisy_graph,'coord').values())
# noisy_graph_coord[:5]

In [18]:
ground_truth, noisy_graph,noisy_coord = generate_noisy_graph_3(G,len(original_coord),4,10,0)

nb_outliers:  4
nb_outliers:  4
all nodes:  24
{20: 9, 23: 13, 21: 10, 22: 16, 9: 20, 13: 23, 10: 21, 16: 22}


In [326]:
len(ground_truth)

20

In [327]:
len(noisy_graph.nodes)

25

In [328]:
noisy_graph.nodes

NodeView((0, 1, 2, 20, 4, 5, 22, 7, 8, 9, 24, 11, 12, 13, 14, 15, 16, 17, 18, 19, 3, 21, 6, 23, 10))

In [329]:
index = 0
for i in ground_truth:
    print(noisy_graph.nodes[i]['coord'],noisy_coord[index])
    index+=1

[ 89.37296722 -43.42786467 -11.24692408] [ 89.37296722 -43.42786467 -11.24692408]
[-70.68124033 -61.88920231  34.26206215] [-70.68124033 -61.88920231  34.26206215]
[53.24682498 29.71840425 79.2564955 ] [53.24682498 29.71840425 79.2564955 ]
[ 72.84140935 -64.61771082  22.7745589 ] [ 72.84140935 -64.61771082  22.7745589 ]
[-59.42440855  -2.15333201 -80.39964446] [-59.42440855  -2.15333201 -80.39964446]
[ -4.03898414 -96.10700753 -27.33367358] [ -4.03898414 -96.10700753 -27.33367358]
[-98.70326713 -15.96603748  -1.65852511] [-98.70326713 -15.96603748  -1.65852511]
[ 39.11551637  80.70289667 -44.23820575] [ 39.11551637  80.70289667 -44.23820575]
[-68.16044975 -71.4933573  -15.58373994] [-68.16044975 -71.4933573  -15.58373994]
[21.49964697 94.08992727 26.16965353] [21.49964697 94.08992727 26.16965353]
[-18.26719043 -94.98477029  25.38115771] [-18.26719043 -94.98477029  25.38115771]
[-17.9530495   96.09616462 -21.0526758 ] [-17.9530495   96.09616462 -21.0526758 ]
[-20.62593061  90.47712897 -

In [49]:
compute_noisy_edges = tri_from_hull(noisy_coord_3+list(outlier_coords))

In [130]:
ground_truth_sequence = []

coords_new = compute_noisy_edges.vertices.view(np.ndarray)

key = []
value = []

for ar1 in noisy_coord_3:
    for i in range(len(coords_new)):
        if np.mean(ar1) == np.mean(coords_new[i]):
            if i >=20:
                key.append(i)
                print("key:",i)
            ground_truth_sequence.append(i)
            
            
for outlier in outlier_coords:
    for i in range(len(coords_new)):
        if np.mean(coords_new[i]) == np.mean(outlier):
            if i<20:
                value.append(i)
                print("value:",i)
                
index = 0
for j in range(len(ground_truth_sequence)):
    if ground_truth_sequence[j] == key[index]:
        ground_truth_sequence[j] = value[index]
        index+=1
                
mapping = dict(zip(key,value))

noisy_graph_3 = nx.relabel_nodes(noisy_graph_3, mapping)

key: 21
key: 23
key: 20
key: 22
value: 6
value: 10
value: 2
value: 0


In [131]:
ground_truth_sequence

[5, 11, 7, 9, 12, 1, 4, 6, 10, 15, 8, 17, 19, 13, 18, 2, 16, 3, 14, 0]

In [147]:
noisy_coord_3[19]

array([-35.58280458,  86.04214399,  36.47757498])

In [145]:
noisy_graph_3.nodes[6]

{'coord': array([49.16931926, 81.21885571, 31.39865475])}

In [148]:
test_G.nodes[0]

{'coord': array([-35.58280458,  86.04214399,  36.47757498])}

In [93]:
ground_truth_sequence

[5, 11, 7, 9, 12, 1, 4, 6, 23, 15, 8, 17, 19, 13, 18, 20, 16, 3, 14, 22]

In [142]:
test_G = noisy_graph_3.copy()

test_G = nx.relabel_nodes(test_G, mapping)